# Scenario 3: Customer Segmentation Query


## My Prompt:
You are a SQL expert helping the product team segment users for a feature rollout.

We are working with a database that contains these tables:

- `user_activity` (user_id, last_login_date, feature_usage_count, account_type)
- `transactions` (transaction_id, user_id, transaction_date, amount, platform)
- `user_preferences` (user_id, communication_preference, interface_theme, notification_settings)

I need you to write a SQL query that does the following:

1. Identifies **active users** who have logged in within the last 30 days.
2. Filters those users to find **high-value customers**, defined as the **top 20% by total spending** (across all their transactions).
3. Returns a summary of **user preference trends** (e.g., counts or percentages of interface themes, notification settings, etc.) among those high-value users.

Please:
- Use proper or subqueries for clarity and efficiency
- Use **percentile calculation**  to find the top 20%
- Format the output in a readable form with grouped preference stats
- Include comments and assume the current date is available via `CURRENT_DATE`

Let me review and test the results before finalizing.


In [ ]:
# Generated code:
#Step 1: Identify Active Users and Calculate Total Spend
WITH active_users AS (
    SELECT user_id
    FROM user_activity
    WHERE last_login_date >= CURRENT_DATE - INTERVAL '30 days'
),

user_spending AS (
    SELECT 
        t.user_id,
        SUM(t.amount) AS total_spend
    FROM transactions t
    JOIN active_users a ON t.user_id = a.user_id
    GROUP BY t.user_id
),

#Step 2: Calculate the 80th percentile spend threshold
spend_threshold AS (
    SELECT PERCENTILE_CONT(0.8) WITHIN GROUP (ORDER BY total_spend) AS threshold
    FROM user_spending
),

#Step 3: Get High-Value Users
high_value_users AS (
    SELECT u.user_id
    FROM user_spending u
    JOIN spend_threshold s ON u.total_spend >= s.threshold
)

#Step 4 & 5: Summarize Preferences of High-Value Users
SELECT 
    up.interface_theme,
    COUNT(*) AS theme_count,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS theme_percentage
FROM user_preferences up
JOIN high_value_users hv ON up.user_id = hv.user_id
GROUP BY up.interface_theme

UNION ALL

SELECT 
    up.notification_settings,
    COUNT(*) AS notification_count,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS notification_percentage
FROM user_preferences up
JOIN high_value_users hv ON up.user_id = hv.user_id
GROUP BY up.notification_settings

UNION ALL

SELECT 
    up.communication_preference,
    COUNT(*) AS communication_count,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS communication_percentage
FROM user_preferences up
JOIN high_value_users hv ON up.user_id = hv.user_id
GROUP BY up.communication_preference;


# Follow-up Prompts:
- "Calculate total spend per user and filter to the top 20% using percentile logic."
- "Summarize user preferences for high-value users with counts and percentages."
- "Break down interface theme, notification settings, and communication preferences separately."
- "Include comments in the SQL to clarify each step."
- "Use UNION ALL to combine results into one output while keeping preference types identifiable."

In [ ]:
#Final Solution
-- Step 1: Get active users who logged in within the last 30 days
WITH active_users AS (
    SELECT user_id
    FROM user_activity
    WHERE last_login_date >= CURRENT_DATE - INTERVAL '30 days'
),

-- Step 2: Calculate total spend per active user
user_spending AS (
    SELECT 
        t.user_id,
        SUM(t.amount) AS total_spend
    FROM transactions t
    JOIN active_users a ON t.user_id = a.user_id
    GROUP BY t.user_id
),

-- Step 3: Determine the 80th percentile spend threshold
spend_threshold AS (
    SELECT 
        PERCENTILE_CONT(0.8) WITHIN GROUP (ORDER BY total_spend) AS threshold
    FROM user_spending
),

-- Step 4: Filter to high-value users (top 20% spenders)
high_value_users AS (
    SELECT 
        u.user_id
    FROM user_spending u
    JOIN spend_threshold s ON u.total_spend >= s.threshold
)

-- Step 5: Summarize user preferences by type and combine using UNION ALL
-- Interface Theme Preferences
SELECT 
    'interface_theme' AS preference_type,
    up.interface_theme AS preference_value,
    COUNT(*) AS count,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS percentage
FROM user_preferences up
JOIN high_value_users hv ON up.user_id = hv.user_id
GROUP BY up.interface_theme

UNION ALL

-- Notification Settings Preferences
SELECT 
    'notification_settings' AS preference_type,
    up.notification_settings AS preference_value,
    COUNT(*) AS count,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS percentage
FROM user_preferences up
JOIN high_value_users hv ON up.user_id = hv.user_id
GROUP BY up.notification_settings

UNION ALL

-- Communication Preferences
SELECT 
    'communication_preference' AS preference_type,
    up.communication_preference AS preference_value,
    COUNT(*) AS count,
    ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS percentage
FROM user_preferences up
JOIN high_value_users hv ON up.user_id = hv.user_id
GROUP BY up.communication_preference;


## Scenario 3: Discussion Questions

### 1. How were percentile calculations handled?

The AI used the `PERCENTILE_CONT` function with `WITHIN GROUP (ORDER BY ...)` to calculate the 80th percentile of total spending. This approach accurately identifies high-value users based on a continuous distribution rather than ranking users by buckets. Earlier versions suggested `NTILE`, but using `PERCENTILE_CONT` is more precise when the goal is to apply a fixed threshold.

---

### 2. What approaches to date filtering were used?

The AI correctly filtered active users using the condition `last_login_date >= CURRENT_DATE - INTERVAL '30 days'`. This ensures only users who have logged in within the past month are included. The use of `CURRENT_DATE` makes the query dynamic and portable. This logic was correctly implemented in the first attempt.

---

### 3. How was the query optimized (CTE, subquery, etc.)?

The AI used Common Table Expressions (CTEs) to structure the query in modular steps. Each major task — filtering active users, calculating total spend, determining the percentile threshold, and selecting high-value users — was broken into its own CTE. This made the query more readable, easier to debug, and reusable for future extensions. UNION ALL was used to consolidate different preference summaries into a single output while preserving context.
